# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix
from utils_ import reduce_mem_usage,freq_enc
from sklearn.inspection import permutation_importance
# Матричная факторизация
import catboost
import lightgbm
from implicit import als
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
# Модель второго уровня
from lightgbm import LGBMClassifier
from implicit.bpr import BayesianPersonalizedRanking
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
import shap
# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [3]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [4]:
data = pd.read_csv('../data/retail_train.csv')
item_features = pd.read_csv('../data/product.csv')
user_features = pd.read_csv('../data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [8]:
recommender = MainRecommender(data_train_lvl_1)

In [19]:
recommender.get_als_recommendations(1, N=5)

[962615, 883616, 841584, 5577022, 1120261]

In [20]:
recommender.get_own_recommendations(1, N=5)

[856942, 9297615, 5577022, 877391, 9655212]

In [21]:
recommender.get_similar_items_recommendation(0, N=5)

[1029743, 1106523, 5569230, 916122, 844179]

In [22]:
recommender.get_similar_users_recommendation(123, N=5)

[6424460, 6424460, 6424460, 6424460, 6424460]

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [13]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index().iloc[1:,:]
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."
2,4,"[883932, 970760, 1035676, 1055863, 1097610, 67..."


In [14]:
# your_code

train_users = list(set(data_train_lvl_1['user_id'].unique()) & set(data_val_lvl_1['user_id'].unique()) & set(data_val_lvl_2['user_id'].unique()))
result_lvl_1 = result_lvl_1[result_lvl_1['user_id'].isin(train_users)]
k=50
result_lvl_1['als'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=k))
result_lvl_1['own'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=k))
result_lvl_1['similar_items'] = result_lvl_1['user_id'].\
apply(lambda x: recommender.get_similar_items_recommendation(x, N=k))
result_lvl_1['similar_users'] = result_lvl_1['user_id'].\
apply(lambda x: recommender.get_similar_users_recommendation(x, N=k))

for col in result_lvl_1.columns[2:]:
    print(col,': ',result_lvl_1.apply(lambda row: recall_at_k(row[col], row['actual'], k=5), axis=1).mean() )

als :  0.012183225429171989
own :  0.017557549929729955
similar_items :  0.004838860044394377
similar_users :  0.005412182133251817


In [1]:
# own дает лучший результат

In [11]:
# your_code

result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index().iloc[1:,:]
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

train_users = list(set(data_train_lvl_1['user_id'].unique()) & set(data_val_lvl_1['user_id'].unique()) & set(data_val_lvl_2['user_id'].unique()))
result_lvl_1 = result_lvl_1[result_lvl_1['user_id'].isin(train_users)]

K = [20, 50, 100, 200, 500]

for k in K:
    
    result_lvl_1[f'als_{k}'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=k))

for col in result_lvl_1.columns[2:]:
    
    print(col,': ',result_lvl_1.apply(lambda row: recall_at_k(row[col], row['actual'], k=k), axis=1).mean() )


als_20 :  0.02875458520796152
als_50 :  0.04716869186696626
als_100 :  0.06892605668077262
als_200 :  0.09723846711056683
als_500 :  0.147329699381764


In [16]:
# оптимальный вариант als_100

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [15]:
# your_code

# Реализовано в рамках курсового проекта